In [6]:
from os import fsencode, listdir, fsdecode
from PIL import Image, ImageDraw

directory = fsencode('images')
for file in listdir(directory)[:1]:
    filename = fsdecode(file)
    img = Image.open('images/' + filename)
    # Sample 10 pixels from where there should always be solid background
    samples = 0
    for i in range(1000, 1010):
        samples += sum(img.getpixel((i, 100))) / 3
    sample_average = samples / 10
    # print('Sample average is {}'.format(sample_average))
    center_x = int(img.width / 2)
    center_y = int(img.height / 2)
    upper_edge_y = None
    for i in range(0, img.height):
        average_pixel_rgb = sum(img.getpixel((center_x, i))) / 3
        # print (average_pixel_rgb)
        if not (sample_average * 0.95 < average_pixel_rgb < sample_average * 1.05):
            # We have likely hit the upper edge of the real image
            draw = ImageDraw.Draw(img)
            draw.line((0, i, img.width, i), fill=128)
            # img.show()
            upper_edge_y = i
            break
    
    lower_edge_y = None
    for i in range(img.height - 1, 0, -1):
        average_pixel_rgb = sum(img.getpixel((center_x, i))) / 3
        # print (average_pixel_rgb)
        if not (155 < average_pixel_rgb < 170):
            # We have likely hit the lower edge of the real image
            draw = ImageDraw.Draw(img)
            draw.line((0, i, img.width, i), fill=128)
            # img.show()
            lower_edge_y = i
            break
            
    right_edge_x = None
    for i in range(img.width - 1, 0, -1):
        average_pixel_rgb = sum(img.getpixel((i, center_y))) / 3
        # print (average_pixel_rgb)
        if not (155 < average_pixel_rgb < 170):
            # We have likely hit the left edge of the real image
            draw = ImageDraw.Draw(img)
            draw.line((i, 0, i, img.height), fill=128)
            # img.show()
            right_edge_x = i
            break
    
    img.show()
            
    # TODO: Find where the sciency colorgram-thing ends and left gray area begins, from there do regular detection
